In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

project_root = Path.cwd().parent.parent  # Assuming notebook is in experiments/notebooks/
sys.path.insert(0, str(project_root))
# print(sys.path)
from experiments.analysis_utils.data_processing import read_directory

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Read the data
path_to_runs = Path("../../results/simulations/actors_labelling/")
# print(path_to_runs.resolve())
# print(path_to_runs.resolve().exists())
proposal_df_full, start_data_df_full, timestep_data_df_full = read_directory(path_to_runs.resolve())

Assume that we sweep over counter_label_percentage. It is how it is called in the start_data_df. Then the cell below should work


In [ ]:
from experiments.analysis_utils.metrics import analyze_veto_timing_by_seals

veto_stats = analyze_veto_timing_by_seals(
    timestep_data_df_full, start_data_df_full, additional_columns=("determining_factor",)
)
veto_stats["attack_success_rate"] = 100 - veto_stats["veto_rate"]
veto_stats["n_successful_attacks"] = veto_stats["attack_success_rate"] / 100 * veto_stats["total_runs"]
print("Veto statistics by seal parameters:")
display(veto_stats)

Veto statistics by seal parameters:


veto_rate  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor               
0.01                         0.1                           40                    0.377358   
                                                           43                    1.320755   
                                                           46                    5.454545   
                                                           49                   14.545455   
                                                           52                   31.616162   
                                                           55                   53.939394   
                                                           58                   74.066986   
                                                           61                   89.473684   
                                                           64                   95.789474   
                                                           67                   98.373206   
                                                           70                   99.081633   
                                                           73                   99.897959   
                                                           76                  100.000000   

                                                                               mean_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor                      
0.01                         0.1                           40                          38.500000   
                                                           43                          37.142857   
                                                           46                          36.518519   
                                                           49                          36.055556   
                                                           52                          35.584665   
                                                           55                          34.807116   
                                                           58                          34.034884   
                                                           61                          32.994652   
                                                           64                          31.928072   
                                                           67                          30.968872   
                                                           70                          30.178167   
                                                           73                          29.481103   
                                                           76                          28.925510   

                                                                               median_time_to_veto  \
first_seal_rage_quit_support second_seal_rage_quit_support determining_factor                        
0.01                         0.1                           40                                 38.0   
                                                           43                                 37.0   
                                                           46                                 36.0   
                                                           49                                 36.0   
                                                           52                                 35.0   
                                                           55                                 35.0   
                                                           58                                 34.0   
                                                           61                                 33.0   
                                                           64                                 32.0   
                                                           67

In [ ]:
import numpy as np

np.arange(40, 79, 3)
[40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76]

[40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76]

Now what experiments should we do? I propose binary search approach. Assume we want to find counter_label_percentage=H such that veto_rate is 0%. We know that at H=0 veto_rate is 100%. We also know that at H=100 veto_rate is 0%. You've already tried H=50 and veto_rate was >0%. So we should try H=75. If veto_rate at H=75 is still >0% we should try H=87.5. And so on until we find the value of H where veto_rate is 0%.

Similarly, we can find the value of L such that veto_rate is 100%. We know that at L=0 veto_rate is 100%, and at L=50 veto_rate is <100%. So we should try L=25. If veto_rate at L=25 is still <100% we should try L=12.5. And so on until we find the value of L where veto_rate is 100%.
